# Part 03: Training Pipeline


## 🗒️ Este notebook está dividido em 3 seções principais:
1. Feature Selection.
2. Pré-processamento de dados.
3. Criação de conjuntos de dados de treinamento.
4. Carregamento dos dados de treinamento.
5. Treinamento do modelo.
6. Registrar o modelo no registro de modelos do Hopsworks.

![02_training-dataset](../../images/02_training-dataset.png)

## 📝 Importando os Pacotes

In [ ]:
# Importando pacotes
import joblib
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import xgboost as xgb

from sklearn.metrics import r2_score

# Mutando warnings
import warnings
warnings.filterwarnings('ignore')

---

## 📡 Conectando ao Hopsworks Feature Store

In [ ]:
# Importando os pacotes
import hopsworks

# Login na Hopsworks
project = hopsworks.login()

# Carregando o Feature Store
fs = project.get_feature_store()

In [ ]:
# Recuperando os Feature Groups
citibike_usage_fg = fs.get_or_create_feature_group(
    name="citibike_usage",
    version=1,
)

citibike_stations_info_fg = fs.get_or_create_feature_group(
    name="citibike_stations_info",
    version=1,
)

us_holidays_fg = fs.get_or_create_feature_group(
    name="us_holidays",
    version=1,
)

meteorological_measurements_fg = fs.get_or_create_feature_group(
    name="meteorological_measurements",
    version=1,
)

---

## 🖍 Criação e Recuperação da Feature View

Vamos começar selecionando todas as features que você deseja incluir para treinamento/inferência do modelo.

In [ ]:
# Selecionar features para os dados de treinamento.
query = meteorological_measurements_fg.select_except(["timestamp"])\
                          .join(
                                us_holidays_fg.select_except(["timestamp"]),
                                on="date", join_type="left"
                          )\
                          .join(
                              citibike_usage_fg.select_except(["timestamp"]),
                              on="date", join_type="left"
                          )


In [ ]:
# # Remova o comentário e execute a célula abaixo se quiser ver algumas linhas desta consulta
# # mas você terá que esperar algum tempo

# query.read()


`Feature Views` fica entre **Feature Groups** e **Conjuntos de Dados de Treinamento**. Combinando **Feature Groups**, podemos criar **Feature Views** que armazenam metadados de nossos dados. Tendo **Feature Views**, podemos criar **Conjuntos de Dados de Treinamento**.

As Feature Views permitem esquemas na forma de uma consulta com filtros, definem uma feature/label de destino do modelo e funções de transformação adicionais.

Para criar uma Feature View, podemos usar o método `FeatureStore.get_or_create_feature_view()`.

Podemos especificar os seguintes parâmetros:

- `name` - nome do grupo de características.

- `version` - versão do grupo de características.

- `labels` - nossa variável alvo.

- `transformation_functions` - funções para transformar nossas características.

- `query` - objeto de consulta com dados.

In [ ]:
# Criando o Feature View
feature_view = fs.get_or_create_feature_view(
    name='citibike_fv',
    query=query,
    labels=["users_count"],
    version=1,   
)

---

## 🏋️ Criando o Dataset de Treino

No Hopsworks, o conjunto de dados de treinamento é uma query em que a projeção (conjunto de features) é determinada pela `FeatureView` parent, com uma opção de snapshot no disco dos dados retornados pela consulta.

**O conjunto de dados de treinamento pode conter divisões como:** 
* Conjunto de treinamento - o subconjunto de dados de treinamento usado para treinar um modelo.
* Conjunto de validação - o subconjunto de dados de treinamento usado para avaliar hparams ao treinar um modelo.
* Conjunto de teste - o subconjunto de dados de treinamento retido usado para avaliar um modelo.

Para criar um conjunto de dados de treinamento, você usará o método `FeatureView.train_test_split()`.

Aqui estão algumas coisas importantes:

- Ele herda o nome da `FeatureView`.

- O `feature store` atualmente suporta os seguintes formatos de dados para conjuntos de dados de treinamento: **tfrecord, csv, tsv, parquet, avro, orc**.

- Você pode escolher o formato desejado usando o parâmetro **data_format**.

- **start_time** e **end_time** para filtrar o conjunto de dados em um intervalo de tempo específico.

- Você pode criar divisões **treino, teste** usando `train_test_split()`.

- Você pode criar divisões **treino, validação, teste** usando os métodos `train_validation_test_splits()`.

- A única coisa é que devemos especificar a proporção desejada das divisões.

In [ ]:
# Separando os conjuntos em treino e teste
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    train_start="2023-01-01",
    train_end="2023-05-01",
    test_start="2023-05-02",
    test_end="2023-05-31",
)

In [ ]:
# Converter as colunas especificadas no conjunto de treinamento para o tipo float
X_train.iloc[:, 1:-1] = X_train.iloc[:, 1:-1].astype(float)

# Converter as colunas especificadas no conjunto de teste para o tipo float
X_test.iloc[:, 1:-1] = X_test.iloc[:, 1:-1].astype(float)

print(f'⛳️ Formato de X_train: {X_train.shape}')
print(f'⛳️ Formato de y_train: {y_train.shape}')


In [ ]:
# Definir o índice de vários níveis para o conjunto de treinamento usando as colunas 'date' e 'station_id'
X_train = X_train.set_index(["date", "station_id"])

# Definir o índice de vários níveis para o conjunto de teste usando as colunas 'date' e 'station_id'
X_test = X_test.set_index(["date", "station_id"])

# Remover linhas com valores ausentes no conjunto de treinamento
X_train.dropna(inplace=True)

# Remover linhas com valores ausentes no conjunto de teste
X_test.dropna(inplace=True)

# Remover linhas com valores ausentes nos rótulos de treinamento
y_train.dropna(inplace=True)

# Remover linhas com valores ausentes nos rótulos de teste
y_test.dropna(inplace=True)

# Exibir as três primeiras linhas do conjunto de treinamento
X_train.head(3)


---
## 🧬 Machine Learning

In [ ]:
# Criar um Regressor XGBoost
regressor = xgb.XGBRegressor()

# Treinar o modelo usando o conjunto de treinamento
regressor.fit(X_train, y_train)


In [ ]:
# Fazer previsões usando o modelo XGBoost treinado
y_pred = regressor.predict(X_test)

# Calcular e mostrar o R2 Score para o modelo XGBoost
r2_xgb = r2_score(y_pred, y_test.values)
print("🎯 R2 Score para o modelo XGBoost:", r2_xgb)


In [ ]:
# Criar um DataFrame com valores reais e previstos
df_ = pd.DataFrame({
    "y_true": np.hstack(y_test.values),
    "y_pred": y_pred,
})

# Criar um gráfico de resíduos usando o Seaborn
residplot = sns.residplot(data=df_, x="y_true", y="y_pred", color='#613F75')

# Definir títulos e rótulos do gráfico
plt.title('Resíduos do Modelo')
plt.xlabel('Observação #')
plt.ylabel('Erro')

# Mostrar o gráfico
plt.show()


In [ ]:
# Plotando o residual
fig = residplot.get_figure()


---
### ⚙️ Esquema do Modelo

O modelo precisa ser configurado com um [Esquema do Modelo](https://docs.hopsworks.ai/3.0/user_guides/mlops/registry/model_schema/), que descreve as entradas e saídas para um modelo.

Um Esquema do Modelo pode ser gerado automaticamente a partir de exemplos de treinamento, como mostrado abaixo.

In [ ]:
# Importando os pacotes necessários
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Criar esquemas de entrada e saída usando os dados de treinamento fornecidos
input_schema = Schema(X_train)
output_schema = Schema(y_train)

# Criar um esquema de modelo com os esquemas de entrada e saída
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

# Converter o esquema do modelo para um dicionário
model_schema.to_dict()


## 🗄 Registro de Modelos

Uma das funcionalidades no Hopsworks é o registro de modelos (*Model Registry*). É aqui que você pode armazenar diferentes versões de modelos e comparar seu desempenho. Modelos do registro podem então ser disponibilizados como pontos de Endpoints de API.

In [ ]:
# Criar um diretório para o modelo se ele não existir
model_dir = "citibike_xgb_model"
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

# Salvar o modelo de regressão XGBoost no diretório especificado
joblib.dump(regressor, model_dir + '/citibike_xgb_model.pkl')

# Salvar o plot do gráfico de resíduos como uma imagem no diretório do modelo
fig.savefig(model_dir + "/residplot.png")


In [ ]:
# Obter o registro de modelos para o projeto
mr = project.get_model_registry()

# Criar um modelo Python no registro de modelos
citibike_model = mr.python.create_model(
    name="citibike_xgb_model", 
    metrics={"r2_score": r2_xgb},
    model_schema=model_schema,
    input_example=X_train.sample(), 
    description="Predictor de usuários do Citibike por estação",
)

# Salvar o diretório do modelo no registro de modelos
citibike_model.save(model_dir)


## ⏭️ **Próxima Aula:** Parte 04: Inferência em Lote

No próximo notebook, você usará seu modelo registrado para prever dados em lote.